In [1]:
import requests
import urllib.request, json 
import pandas as p
import altair as alt

In [2]:
# Retrieve all tracks from variantMuts group using UCSC API

data = {}
r = requests.get('https://api.genome.ucsc.edu/list/tracks?genome=wuhCor1').json()
for key in r['wuhCor1']['variantMuts']:
    # This assumes that all relevant tables beging with "variant"
    if key.startswith('variant'):
        data[key] = requests.get('https://api.genome.ucsc.edu/getData/track?genome=wuhCor1;track={}'.format(key)).json()

In [3]:
for key in data.keys():
    print(key)

variantNucMutsV2_B_1_617_1
variantAaMutsV2_B_1_617_1
variantNucMutsV2_B_1_526
variantAaMutsV2_B_1_526
variantNucMutsV2_B_1_525
variantAaMutsV2_B_1_525
variantNucMutsV2_B_1_429
variantAaMutsV2_B_1_429
variantNucMutsV2_B_1_621
variantAaMutsV2_B_1_621
variantNucMutsV2_C_37
variantAaMutsV2_C_37
variantNucMuts_B_1_1_529
variantAaMuts_B_1_1_529
variantNucMutsV2_B_1_617_2
variantAaMutsV2_B_1_617_2
variantNucMutsV2_P_1
variantAaMutsV2_P_1
variantNucMutsV2_B_1_351
variantAaMutsV2_B_1_351
variantNucMutsV2_B_1_1_7
variantAaMutsV2_B_1_1_7


In [4]:
expanded_list = { 'POS':[],'variant':[],'lineage':[] }
for strain in data.keys():
    for variant in data[strain][strain]:
        for i in range(variant['chromStart'],variant['chromEnd']):
            expanded_list['POS'].append(i)
            expanded_list['variant'].append(variant['name'])
            expanded_list['lineage'].append(strain.lstrip('variantNucAaMutsV2_'))

In [5]:
df = pd.DataFrame.from_dict(expanded_list)

In [6]:
alt.renderers.set_embed_options(actions=True)

alt.Chart(df.groupby(['lineage','variant']).agg({'POS':'min'}).reset_index()).mark_tick(color='green').encode(
    x=alt.X("POS:Q",title='Position',axis=alt.Axis(grid=False)),
    y=alt.Y("lineage:O",title='Lineage',
            axis=alt.Axis(grid=False)),
    tooltip=[
            alt.Tooltip("variant",title="variant")]
).properties(
    width=800,
    height=200
)

alt.Chart(...)

In [7]:
df.to_csv('ucsc_lineages.csv.gz',index=False,compression='gzip')